In [2]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.workbook import Workbook
from datetime import datetime

In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt

class Agent:
    def __init__(self, initial_wealth):
        self.money = initial_wealth

    def consume(self, seller, quantity, amount):
        self.money -= amount
        seller.money += amount

In [4]:
class Simulation:
    def __init__(self, num_agents, initial_wealth, num_time_steps, buy_prob):
        self.num_agents = num_agents
        self.initial_wealth = initial_wealth
        self.num_time_steps = num_time_steps
        self.buy_prob = buy_prob
        self.agents = [Agent(initial_wealth) for _ in range(num_agents)]
        self.total_tax = 0
        self.transaction_amounts = []

    def calculate_tax(self, wealth):
        scaled_wealth = wealth

        # Scale tax slabs based on the ratio of max_wealth to 1500000
        scaled_slab_1 = 109
        scaled_slab_2 = 218
        scaled_slab_3 = 327
        scaled_slab_4 = 435
        scaled_slab_5 = 544

        if scaled_wealth <= scaled_slab_1:
            return 0
        elif scaled_wealth <= scaled_slab_2:
            return 0.05 * (scaled_wealth - scaled_slab_1)
        elif scaled_wealth <= scaled_slab_3:
            return 0.1 * (scaled_wealth - scaled_slab_2) + 0.05 * (scaled_slab_2 - scaled_slab_1)
        elif scaled_wealth <= scaled_slab_4:
            return 0.15 * (scaled_wealth - scaled_slab_3) + 0.05 * (scaled_slab_2 - scaled_slab_1) + 0.1 * (scaled_slab_3 - scaled_slab_2)
        elif scaled_wealth <= scaled_slab_5:
            return 0.2 * (scaled_wealth - scaled_slab_4) + 0.05 * (scaled_slab_2 - scaled_slab_1) + 0.1 * (scaled_slab_3 - scaled_slab_2) + 0.15 * (scaled_slab_4 - scaled_slab_3)
        else:
            return 0.3 * (scaled_wealth - scaled_slab_5) + 0.05 * (scaled_slab_2 - scaled_slab_1) + 0.1 * (scaled_slab_3 - scaled_slab_2) + 0.15 * (scaled_slab_4 - scaled_slab_3) + 0.2 * (scaled_slab_5 - scaled_slab_4)


    def update_agents(self):
        random_indices = random.sample(range(self.num_agents), 2)
        agent1, agent2 = self.agents[random_indices[0]], self.agents[random_indices[1]]

        if random.random() < self.buy_prob:
              # Calculate transaction amount based on the formula provided
              transaction_amount = 0.2 * (agent1.money * agent2.money) / (agent1.money + agent2.money)


              agent1.consume(agent2, 1, transaction_amount)

    def deduct_tax(self):
        for agent in self.agents:
          tax=self.calculate_tax(agent.money)
          agent.money -= tax
          self.total_tax+=tax

    def redist(self):
        add_val=self.total_tax/self.num_agents
        for agent in self.agents:
          agent.money+=add_val
        self.total_tax=0

    def run_simulation(self):
        for i in range(self.num_time_steps):
            self.update_agents()        
        self.deduct_tax()
        self.redist()
        


In [ ]:
import numpy as np
num_agents = 100000 
initial_wealth = 500
num_time_steps = 3000000
buy_prob = 0.1
tax_rate = 0.0375
coeff_variation=[]
gini_indices = []

# Initialize an empty list to store wealth distributions
all_wealth_distributions = []

# Initialize lists to store the wealth of specific agents over time
num_random_agents = 25
agent_wealth_over_time = {agent_id: [random.randint(100, 500)] for agent_id in range(num_agents)}

# Run simulation
sim = Simulation(num_agents, initial_wealth, num_time_steps, buy_prob)
for i in range(1000):
    sim.run_simulation()
    print("iteration ", i+1, " complete")
    wealth_distribution = [agent.money for agent in sim.agents]
    all_wealth_distributions.append(wealth_distribution)

    # Store the wealth of specific agents over time
    for agent_id in agent_wealth_over_time:
        agent_wealth_over_time[agent_id].append(sim.agents[agent_id].money)
        
    # Calculate Gini index
    wealth_distribution_sorted = np.sort(wealth_distribution)
    total_wealth = np.sum(wealth_distribution_sorted)
    cumulative_wealth = np.cumsum(wealth_distribution_sorted) / total_wealth
    cumulative_population = np.linspace(0, 1, len(cumulative_wealth))
    area_between_curves = np.trapz(cumulative_wealth, cumulative_population) - (0.5 - np.trapz(cumulative_population, cumulative_population))
    gini_index = area_between_curves / 0.5
    gini_indices.append(gini_index)

# Flatten the list of wealth distributions
flattened_wealth_distributions = [wealth for distribution in all_wealth_distributions for wealth in distribution]

# Plot the wealth distribution curve
plt.figure(figsize=(10, 6))
plt.plot(sorted(flattened_wealth_distributions), color='blue', linewidth=0.5)
plt.title('Wealth Distribution')
plt.xlabel('Agent Index')
plt.ylabel('Wealth')
plt.grid(True)
plt.show()


iteration  1  complete
iteration  2  complete
iteration  3  complete
iteration  4  complete
iteration  5  complete
iteration  6  complete
iteration  7  complete
iteration  8  complete
iteration  9  complete
iteration  10  complete
iteration  11  complete
iteration  12  complete
iteration  13  complete
iteration  14  complete
iteration  15  complete
iteration  16  complete
iteration  17  complete
iteration  18  complete
iteration  19  complete
iteration  20  complete
iteration  21  complete
iteration  22  complete
iteration  23  complete
iteration  24  complete
iteration  25  complete
iteration  26  complete
iteration  27  complete
iteration  28  complete
iteration  29  complete
iteration  30  complete
iteration  31  complete
iteration  32  complete
iteration  33  complete
iteration  34  complete
iteration  35  complete
iteration  36  complete
iteration  37  complete
iteration  38  complete
iteration  39  complete
iteration  40  complete
iteration  41  complete
iteration  42  complete
i

In [ ]:
# Calculate the mean and standard deviation of the Gini indices
gini_index_mean = np.mean(gini_indices)
gini_index_std = np.std(gini_indices)

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Run ID': range(1, len(gini_indices) + 1),
    'Gini Index': gini_indices,
    'Mean Gini Index': gini_index_mean,
    'Std Gini Index': gini_index_std
})

excel_file = load_workbook('results.xlsx')
writer = pd.ExcelWriter('results.xlsx', engine='openpyxl')
writer.book = excel_file

# Write the DataFrame to the Excel file
results_df.to_excel(writer, sheet_name=f'Run {datetime.now().strftime("%Y-%m-%d %H-%M-%S")}', index=False)

# Save the Excel file
writer.save()
writer.close()

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown widget
agent_dropdown = widgets.Dropdown(
    options=[f'Agent {agent_id}' for agent_id in agent_wealth_over_time.keys()],
    value=f'Agent {next(iter(agent_wealth_over_time.keys()))}',  # Default value
    description='Agent:',
    disabled=False,
)

# Create a function to update the plot based on the selected agent
def update_plot(agent):
    fig.clf()  # Clear the current figure
    ax = fig.add_subplot(111)  # Add a new subplot
    ax.plot(range(len(agent_wealth_over_time[int(agent.split()[-1])])), agent_wealth_over_time[int(agent.split()[-1])], color='blue', alpha=0.5, label=agent)
    ax.set_title(agent)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Wealth')
    ax.grid(True)
    plt.tight_layout()
    plt.show()

# Display the dropdown widget
display(agent_dropdown)

# Update the plot when the dropdown value changes
agent_dropdown.observe(update_plot, names='value')